In [1]:
push!(LOAD_PATH, "..")
using ASE, Potentials, MatSciPy, AtomsInterface, TestAtoms, TightBinding
using PyPlot
import AtOptim
import AtPrecon

In [ ]:
# reference configuration
at = bulk("Al"; cubic=true)
at = repeat(at, (5, 5, 1))
set_pbc!(at, (false, false, false))
X0 = positions(at);
# jiggle the positions
set_positions!(at, X0+0.01 * rand(size(X0)))
# create a calculator
r0 = rnn("Al")
rcut = 2.5 * r0
# calc = MatSciPy.PairCalculator( SWCutoff( LennardJonesPotential(r0, 1.0), rcut, 1.0 ) )
calc = TightBinding.ToyTB.ToyTBModel(r0=r0, rcut=rcut)
TestAtoms.test_potentialenergy(calc, at)
;

-----------------------------
  p | error 
----|------------------------
  2 | 2.5104853e-03 
  3 | 2.5085740e-04 
  4 | 2.5083939e-05 
  

In [ ]:
reload("AtPrecon")
reload("AtOptim")
at = repeat(bulk("Al"; cubic=true), (20,20,3))
println(" #at = ", length(at))
# precon = AtPrecon.IdPrecon()
precon = AtPrecon.ExpPrecon(at, calc)
X0 = positions(at)
srand(12345)
set_positions!(at, X0+0.05 * rand(size(X0)))
@time AtOptim.minimise!(at, calc, 1e-4; precon=precon, disp=3, maxnit=100);

 #at = 4800